In [1]:
import obi_one as obi
import os
import pandas as pd

/Users/pokorny/JupyterLab/git/obi-one/.venv/lib/python3.12/site-packages/conntility/circuit_models/input_spikes.py:22: SyntaxWarning: invalid escape sequence '\s'
  s = pandas.read_csv(spk["spike_file"], sep="\s+").rename(columns={"/scatter": "node_id"})["node_id"]


In [2]:
input_root = "/Users/pokorny/Data/Circuits"
output_root = "../../../obi-output/extracted_tiny_circuits"

In [3]:
# # Neuron IDs for nbS1-HEX0
# N_ids = 10
# node_selection_files = ["./selection1.csv",
#                         "./selection2.csv"]
# def sel_first_n(elements, n):
#     assert len(elements) >= n, f"ERROR: Less than {n} elements to select from!"
#     return elements[:n]
# node_id_lists = [obi.NamedTuple(name=f"N_{N_ids}__" + os.path.splitext(os.path.split(_f)[1])[0], elements=tuple(sel_first_n(pd.read_csv(_f)["node_ids"].to_list(), N_ids))) for _f in node_selection_files]

In [16]:
# Neuron IDs for rCA1-CYLINDER-REF
node_id_lists = [# obi.NamedTuple(name="N_3__rCA1-FFInh-Sel1", elements=(1146, 5684)),  # + virtual 64063
                 obi.NamedTuple(name="N_3__rCA1-ThetaOsc-Sel1", elements=(3903, 5712, 6026))]

In [17]:
circuit_extractions_form = obi.CircuitExtractions(
                    initialize=obi.CircuitExtractions.Initialize(
                        # circuit=obi.Circuit(name="ToyCircuit-S1-6k", path=os.path.join(input_root, "ToyCircuit-S1-6k", "circuit_config.json")),
                        # circuit=obi.Circuit(name="nbS1-HEX0", path=os.path.join(input_root, "nbS1-HEX0", "circuit_config.json")),
                        circuit=obi.Circuit(name="rCA1-CYLINDER-REF", path=os.path.join(input_root, "rCA1-CYLINDER-REF", "circuit_config.json")),
                        run_validation=False,
                    ),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Excitatory", "Inhibitory"], random_sample=[100, 500]),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Layer2", "Layer3", "Layer4", "Layer5", "Layer6"], random_sample=10),
                    neuron_set=obi.IDNeuronSet(neuron_ids=node_id_lists)
)

grid_scan = obi.GridScan(form=circuit_extractions_form, output_root=output_root, coordinate_directory_option="VALUE")

In [18]:
# Display the intermediary data
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
grid_scan.coordinate_instances(display=True)

[2025-06-04 11:50:24,707] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-06-04 11:50:24,709] INFO: neuron_set.neuron_ids: [N_3__rCA1-ThetaOsc-Sel1]
[2025-06-04 11:50:24,711] INFO: 
COORDINATE PARAMETERS
[2025-06-04 11:50:24,712] INFO: neuron_set.neuron_ids: N_3__rCA1-ThetaOsc-Sel1
[2025-06-04 11:50:24,713] INFO: 
COORDINATE INSTANCES
[2025-06-04 11:50:24,714] INFO: CircuitExtraction(idx=0, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'neuron_ids'], value=N_3__rCA1-ThetaOsc-Sel1)], nested_coordinate_subpath_str=PosixPath('.')), type='CircuitExtraction', neuron_set=IDNeuronSet(type='IDNeuronSet', simulation_level_name=None, random_sample=None, random_seed=0, neuron_ids=N_3__rCA1-ThetaOsc-Sel1), initialize=Initialize(type='CircuitExtractions.Initialize', circuit=Circuit(type='Circuit', nam

[CircuitExtraction(idx=0, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'neuron_ids'], value=N_3__rCA1-ThetaOsc-Sel1)], nested_coordinate_subpath_str=PosixPath('.')), type='CircuitExtraction', neuron_set=IDNeuronSet(type='IDNeuronSet', simulation_level_name=None, random_sample=None, random_seed=0, neuron_ids=N_3__rCA1-ThetaOsc-Sel1), initialize=Initialize(type='CircuitExtractions.Initialize', circuit=Circuit(type='Circuit', name='rCA1-CYLINDER-REF', path='/Users/pokorny/Data/Circuits/rCA1-CYLINDER-REF/circuit_config.json'), run_validation=False))]

In [19]:
grid_scan.execute(processing_method='run')

Extracting subcircuit from 'rCA1-CYLINDER-REF'
Copying morphologies for population 'hippocampus_neurons' (3)


Copying .swc morphologies: 100%|█████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 289.27it/s]

Copying 3 biophysical neuron models (.hoc) for population 'hippocampus_neurons' (3)
Extraction DONE
[2025-06-04 11:50:40,045] INFO: create_bbp_workflow_campaign_config() not yet complete.


{0: None}

In [25]:
# Check circuits
from bluepysnap import Circuit

for _inst in grid_scan.coordinate_instances():
    cfg = grid_scan.output_root / str(_inst.neuron_set.neuron_ids) / "circuit_config.json"
    c = Circuit(cfg)
    # print(f"{_inst.neuron_set.neuron_ids}: {c.nodes['S1nonbarrel_neurons'].size} neurons, {c.edges["S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"].size} synapses")
    print(f"{_inst.neuron_set.neuron_ids}: {c.nodes['hippocampus_neurons'].size} neurons, {c.edges["hippocampus_neurons__hippocampus_neurons__chemical_synapse"].size} synapses")


N_3__rCA1-ThetaOsc-Sel1: 3 neurons, 66 synapses
